In [2]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [3]:
def load_nifti_files(file_paths):
    data = []
    for file_path in file_paths:
        img = nib.load(file_path)
        data.append(img.get_fdata())
    return np.array(data)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# Load the MRI scans and labels
labels = [0, 1, 0, 0]
mri_file_paths = ["/content/drive/MyDrive/Personal Files/Projects/Alzheimer's Research/Oasis 2/1.hdr", "/content/drive/MyDrive/Personal Files/Projects/Alzheimer's Research/Oasis 2/2.hdr", "/content/drive/MyDrive/Personal Files/Projects/Alzheimer's Research/Oasis 2/3.hdr", "/content/drive/MyDrive/Personal Files/Projects/Alzheimer's Research/Oasis 2/4.hdr"]
mri_scans = load_nifti_files(mri_file_paths)
labels = np.array(labels)

# Normalize the MRI scans to have values between 0 and 1
mri_scans = mri_scans / np.max(mri_scans)

print("Shape of MRI scans:", mri_scans.shape)

Shape of MRI scans: (4, 256, 256, 128, 1)


In [11]:
# Preprocessing the data to reduce training load
def resize_image(image, new_shape):
    old_shape = np.array(image.shape)
    ratio = new_shape / old_shape
    return zoom(image, ratio, mode='nearest')

resized_scans = np.array([resize_image(scan, (128, 128, 64, 1)) for scan in mri_scans])
print("Shape of preprocessed MRI scans:", resized_scans.shape)

Shape of preprocessed MRI scans: (4, 128, 128, 64, 1)


In [12]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(resized_scans, labels, test_size=0.2, random_state=42)

# Reshape the data to include the channel dimension (required for CNN)
x_train = x_train.reshape((-1, 128, 128, 64, 1))
x_test = x_test.reshape((-1, 128, 128, 64, 1))

In [14]:
def create_model():
    model = Sequential()
    model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(128, 128, 64, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [15]:
model = create_model()
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

Epoch 1/10
1/1 [==============================] - 20s 20s/step - loss: 0.6854 - accuracy: 1.0000 - val_loss: 88.2500 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 16s 16s/step - loss: 9.7884e-34 - accuracy: 1.0000 - val_loss: 170.0459 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 16s 16s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 250.9157 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 16s 16s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 329.7154 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 16s 16s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 405.5007 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 16s 16s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 477.6295 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 16s 16s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss